## N01 - Point Cloud Representation of Jets

In this notebook, we take a look at the LHC Olympics dataset and create a point cloud representation of it in the form of awkward arrays. A visualization of the main kinematic distributions and the particle clouds are also created.

### Load dataset

- The [R&D Dataset](https://zenodo.org/record/4536377) for LHC Olympics 2020 Anomaly Detection Challenge
- Additional [background sample](https://zenodo.org/records/8370758)
- A total of 100k signal ($W'\rightarrow XY$) and 1M QCD dijet background events
- Signal region defined as $3.3 < m_{jj} < 3.7$ Gev
- The signals are generated with $W'$ mass = 3.5 TeV, $X$ mass = 500 GeV and $Y$ mass = 100 GeV
- The first 3 * 700 = 2100 columns are the $p_T$, $\eta$ and $\phi$ of the constituent particles (up to 700 of them, zero padded)
- The last column is the class label (0 = background, 1 = signal)

In [1]:
import os
from functools import partial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# for nojit
import numba as nb

# for awkward arrays
import awkward as ak
import vector
vector.register_awkward()
# for jet clustering (old method)
import pyjet as pj
from pyjet import cluster, DTYPE_PTEPM
# for jet clustering (new method)
#import fastjet as fj

proj_dir = "/global/cfs/projectdirs/m3246/AnomalyDetection/LHCO/"
save_dir = "/pscratch/sd/c/chlcheng/dataset/anomaly_detection/LHC_Olympics_2020/LHCO_qq"

# print library versions
print(f'awkward version : {ak.__version__}')
print(f'vector version  : {vector.__version__}')
print(f'pyjet version   : {pj.__version__}')

output_format = 'parquet'
ak_methods = {
    'parquet': {
        'save': ak.to_parquet,
        'load': ak.from_parquet
    },
    'feather': {
        'save': ak.to_feather,
        'load': partial(ak.from_feather, memory_map=True)
    }
}
save_fn = ak_methods[output_format]['save']
load_fn = ak_methods[output_format]['load']

awkward version : 2.4.6
vector version  : 1.1.1.post1
pyjet version   : 1.9.0


In [2]:
## the original shuffled dataset
# input_path = os.path.join(proj_dir, "events_anomalydetection.h5")
# dataset ordered by labels
input_path = os.path.join(proj_dir, "events_anomalydetection_v2.h5")
df = pd.read_hdf(input_path)

In [3]:
# just to make sure it is the same file from zenodo
#!md5sum {input_path}

In [4]:
print(f'Number of events: {df.shape[0]}')
df.head()

Number of events: 1100000


,0,1,2,3,4,5,6,7,8,9,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,1.840249,-2.358095,0.368419,1.317952,-2.464229,0.797502,7.523068,-2.425296,0.405914,6.519727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.953231,-2.312420,-0.239617,0.243119,-2.248381,-0.270049,0.427222,-1.736402,-1.701119,0.347634,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.821783,-0.666056,-0.098759,11.534968,-1.580616,2.448051,20.668016,-1.579325,2.474435,13.348019,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.489804,-2.238683,2.117016,0.415852,-0.088672,-2.942330,1.489766,-2.257028,1.326838,1.125934,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.088889,-1.336245,-0.013672,3.380606,-0.996914,-0.043640,1.360203,-0.825792,0.415315,0.983310,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# check that there are 1M background and 100k signal events
print(f'Number of signal events     : {(df[2100] == 1).sum():,}')
print(f'Number of background events : {(df[2100] == 0).sum():,}')

Number of signal events     : 100,000
Number of background events : 1,000,000


### Jet Clustering

In [6]:
from aliad.interface.fastjet import JetClusteringTool
tool = JetClusteringTool(R=1.0, ptmin=20.0)

In [7]:
jet_constituents = {
    "signal"     : tool.get_constituent_arrays_from_padded_data(df[df[2100] == 1].values),
    "background" : tool.get_constituent_arrays_from_padded_data(df[df[2100] == 0].values),
}

In [8]:
# first event, first 5 particles
jet_constituents["signal"][0][:5]

array([(0.35954976, -0.98441672,  1.40178788, 0.),
       (0.35840625, -1.80749536, -0.35914496, 0.),
       (0.87615931, -1.90636575,  0.02573433, 0.),
       (2.16851616, -1.72313857,  0.01530619, 0.),
       (2.2055459 , -1.5062834 ,  0.11419465, 0.)],
      dtype=[('pT', '<f8'), ('eta', '<f8'), ('phi', '<f8'), ('mass', '<f8')])

### Extract jet clusters

In [9]:
# need to process in batches due to high memory demand

save_dir = "/pscratch/sd/c/chlcheng/dataset/anomaly_detection/LHC_Olympics_2020/LHCO_qq"

save_path = {
    "signal"     : os.path.join(save_dir, output_format, "row_based", "batches", "point_cloud_sig_{obj}_batch_{i}." + output_format),
    "background" : os.path.join(save_dir, output_format, "row_based", "batches", "point_bloud_bkg_{obj}_batch_{i}." + output_format)
}
if not os.path.exists(os.path.dirname(save_path['signal'])):
    os.makedirs(os.path.dirname(save_path['signal']))

batchsize = 100000
# maximum number of jets to save the constituent information
max_jet = 4
cache = True
save = True
all_jets = {}
all_constituents = {}
for dtype in ["signal", "background"]:
    all_jets[dtype] = []
    all_constituents[dtype] = {}
    for j in range(max_jet):
        all_constituents[dtype][f'j{j+1}'] = []
    nbatch = len(jet_constituents[dtype]) // batchsize
    if len(jet_constituents[dtype]) % batchsize != 0:
        nbatch += 1    
    for i in range(nbatch):
        i_start = i * batchsize
        i_end  = max((i+1)*batchsize, nbatch)
        path_jets = save_path[dtype].format(i=i+1, obj="jets")
        paths_constituents = []
        for j in range(max_jet):
            path_constituents = save_path[dtype].format(i=i+1, obj=f"constituents_jet_{j+1}")
            paths_constituents.append(path_constituents)
        # only do jet clustering if not cached
        if os.path.exists(path_jets) and all([os.path.exists(path) for path in paths_constituents]) and cache:
            jets_batch = None
        else:
            print(f'Creating clustered jet data for "{dtype}" (batch {i+1} / {nbatch})')
            jets_batch = tool.get_inclusive_jets_array(jet_constituents[dtype][i_start: i_end])
        # convert to awkward array
        if os.path.exists(path_jets) and cache:
            print(f'Cached existing clustered jet data from "{path_jets}"')
            jet_array = load_fn(path_jets)
        else:
            print(f'Converting clustered jet data to awkward arrays')
            jet_array = tool.to_awkward_jets_array(jets_batch)
            if save:
                print(f'Saving clustered jet data to "{path_jets}"')
                save_fn(jet_array, path_jets)
        all_jets[dtype].append(jet_array)
        for j in range(max_jet):
            path_constituents = paths_constituents[j]
            if os.path.exists(path_constituents) and cache:
                print(f'Cached existing clustered jet constituents data from "{path_constituents}"')
                constituent_array = load_fn(path_constituents)
            else:
                print(f'Creating clustered jet constituents data for "{dtype}" (jet {j + 1})')
                constituent_array = tool.to_awkward_constituents_array(jets_batch, jet_index=j)
                if save:
                    print(f'Saving clustered jet constituents data to "{path_constituents}"')
                    save_fn(constituent_array, path_constituents)
            all_constituents[dtype][f'j{j+1}'].append(constituent_array)
    all_jets[dtype] = ak.concatenate(all_jets[dtype])
    for j in range(max_jet):
        all_constituents[dtype][f'j{j+1}'] = ak.concatenate(all_constituents[dtype][f'j{j+1}'])

Cached existing clustered jet data from "/pscratch/sd/c/chlcheng/dataset/anomaly_detection/LHC_Olympics_2020/LHCO_qq/parquet/row_based/batches/point_cloud_sig_jets_batch_1.parquet"
Cached existing clustered jet constituents data from "/pscratch/sd/c/chlcheng/dataset/anomaly_detection/LHC_Olympics_2020/LHCO_qq/parquet/row_based/batches/point_cloud_sig_constituents_jet_1_batch_1.parquet"
Cached existing clustered jet constituents data from "/pscratch/sd/c/chlcheng/dataset/anomaly_detection/LHC_Olympics_2020/LHCO_qq/parquet/row_based/batches/point_cloud_sig_constituents_jet_2_batch_1.parquet"
Cached existing clustered jet constituents data from "/pscratch/sd/c/chlcheng/dataset/anomaly_detection/LHC_Olympics_2020/LHCO_qq/parquet/row_based/batches/point_cloud_sig_constituents_jet_3_batch_1.parquet"
Cached existing clustered jet constituents data from "/pscratch/sd/c/chlcheng/dataset/anomaly_detection/LHC_Olympics_2020/LHCO_qq/parquet/row_based/batches/point_cloud_sig_constituents_jet_4_batc

In [10]:
# save the combined data
save_path = {
    "signal"     : os.path.join(save_dir, output_format, "row_based", "batches", "point_cloud_sig_{obj}." + output_format),
    "background" : os.path.join(save_dir, output_format, "row_based", "batches", "point_cloud_bkg_{obj}." + output_format)
}
for dtype in ['signal', 'background']:
    save_fn(all_jets[dtype], save_path[dtype].format(obj='jets'))
    for j in range(max_jet):
        save_fn(all_constituents[dtype][f'j{j+1}'], save_path[dtype].format(obj=f'constituents_jet_{j+1}'))

In [11]:
print('Jets from first signal event:\n')
all_jets['signal'][0]

Jets from first signal event:



<MomentumArray4D [{pt: 1.91e+03, eta: 0.37, ...}, ...] type='4 * Momentum4D...'>

In [12]:
print('constituent from leading jet of first signal event:\n')
all_constituents['signal']['j1'][0]

constituent from leading jet of first signal event:



<MomentumArray4D [{pt: 1.4, eta: 0.184, ...}, ..., {...}] type='23 * Moment...'>

In [13]:
print('Jets from first background event:\n')
all_jets['background'][0]

Jets from first background event:



<MomentumArray4D [{pt: 1.59e+03, eta: 0.316, ...}, ...] type='4 * Momentum4...'>

In [14]:
print('constituent from leading jet of first background event:\n')
all_constituents['background']['j1'][0]

constituent from leading jet of first background event:



<MomentumArray4D [{pt: 0.576, eta: 0.488, ...}, ...] type='14 * Momentum4D[...'>

### Prepare features for training


Particle-level features

- $p_T$, $p_T^\text{rel}$
- $\eta$, $\Delta \eta$
- $\phi$, $\Delta \phi$
- $\Delta R$

Jet-level features

- $p_T$
- $\eta$
- $\phi$
- $m$
- $N_{\text{particle}}$

In [15]:
features = {}
for dtype in ["signal", "background"]:
    features[dtype] = {}
    for key in all_constituents[dtype]:
        features[dtype][key] = {}
        # jet features
        jet_p4 = ak.sum(all_constituents[dtype][key], axis=-1)
        # convert to TeV
        features[dtype][key]["jet_pt"]   = jet_p4.pt / 1000.
        features[dtype][key]["jet_eta"]  = jet_p4.eta
        features[dtype][key]["jet_phi"]  = jet_p4.phi
        features[dtype][key]["jet_m"]    = jet_p4.m / 1000.
        features[dtype][key]["N"]        = ak.count(all_constituents[dtype][key], axis=-1)
        # particle features
        features[dtype][key]['part_pt']        = all_constituents[dtype][key].pt / 1000.
        features[dtype][key]['part_eta']       = all_constituents[dtype][key].eta
        features[dtype][key]['part_phi']       = all_constituents[dtype][key].phi
        features[dtype][key]['part_e']         = all_constituents[dtype][key].e / 1000.
        features[dtype][key]['part_rel_pt']    = all_constituents[dtype][key].pt / jet_p4.pt
        features[dtype][key]['part_delta_eta'] = all_constituents[dtype][key].deltaeta(jet_p4)
        features[dtype][key]['part_delta_phi'] = all_constituents[dtype][key].deltaphi(jet_p4)
        features[dtype][key]['part_delta_R']   = all_constituents[dtype][key].deltaR(jet_p4)

Fill N-Subjetiness variables

In [16]:
input_path = os.path.join(proj_dir, "events_anomalydetection_v2.features.h5")
df_features = pd.read_hdf(input_path)

In [17]:
df_features.head()

,pxj1,pyj1,pzj1,mj1,tau1j1,tau2j1,tau3j1,pxj2,pyj2,pzj2,mj2,tau1j2,tau2j2,tau3j2,label
0,-1467.239990,611.502014,511.101990,38.896000,8.290660,4.836080,4.260190,1403.579956,-674.551025,-451.670990,237.893997,79.815102,21.010300,16.757601,0.0
1,-1211.239990,347.315002,547.963013,389.532013,191.804001,99.562798,70.872200,619.341003,-62.177299,-1944.040039,22.999201,8.042190,6.335090,5.525360,0.0
2,-1229.619995,649.857971,8.089170,72.155502,47.168098,37.243198,33.658199,1196.250000,-647.896973,-1283.109985,78.230698,15.292900,13.944200,10.013500,0.0
3,-693.304016,-1046.729980,1716.910034,55.797798,24.788601,6.890150,5.813390,747.961975,994.250000,-412.966003,359.113007,175.209000,103.500999,84.447098,0.0
4,-1488.199951,-25.370100,-30.989700,84.891502,26.878799,15.517200,13.260400,1415.640015,20.905100,223.630997,77.506500,57.986000,34.147400,26.660601,0.0


In [18]:
for dtype, label in [("signal", 1), ("background", 0)]:
    df_dtype = df_features[df_features['label'] == label]
    for jtype in ['j1', 'j2']:
        for tau in ['tau1', 'tau2', 'tau3']:
            features[dtype][jtype][f'{tau}'] = df_dtype[f'{tau}{jtype}'].values
        ## can use any of tau1, tau2 or tau3 to check for zeros
        mask = ((features[dtype][jtype][f'tau1'] > 0) & (features[dtype][jtype][f'tau2'] > 0))
        features[dtype][jtype][f'tau12'] = np.where(mask, np.divide(features[dtype][jtype][f'tau2'],
                                                                    features[dtype][jtype][f'tau1'], where=mask), 0)
        features[dtype][jtype][f'tau23'] = np.where(mask, np.divide(features[dtype][jtype][f'tau3'],
                                                                    features[dtype][jtype][f'tau2'], where=mask), 0)
        ## remove events with undefined nsubjetiness ratio
        #for feature in features[dtype][jtype]:
        #    features[dtype][jtype][feature] = features[dtype][jtype][feature][mask]
        #features[dtype][jtype][f'tau12'] = features[dtype][jtype][f'tau2'] / features[dtype][jtype][f'tau1']
        #features[dtype][jtype][f'tau23'] = features[dtype][jtype][f'tau3'] / features[dtype][jtype][f'tau2']

In [20]:
sorted_features = {}
for dtype in ['signal', 'background']:
    sorted_features[dtype] = {'j1':{}, 'j2':{}}
    mass_array = [features[dtype]['j1']['jet_m'],
                  features[dtype]['j2']['jet_m']]
    mass_array = ak.to_numpy(ak.Array(mass_array)).transpose()
    sort_index = np.argsort(-mass_array, axis=1)
    for feature in ['jet_pt', 'jet_eta', 'jet_phi', 'jet_m', 'N', 'tau1', 'tau2', 'tau3', 'tau12', 'tau23']:
        feature_array = [features[dtype]['j1'][feature],
                         features[dtype]['j2'][feature]]
        feature_array = ak.to_numpy(ak.Array(feature_array)).transpose()
        feature_array = np.take_along_axis(feature_array, sort_index, axis=1)
        sorted_features[dtype]['j1'][feature] = np.ascontiguousarray(feature_array[:, 0])
        sorted_features[dtype]['j2'][feature] = np.ascontiguousarray(feature_array[:, 1])
    for feature in ['part_pt', 'part_eta', 'part_phi', 'part_e', 'part_rel_pt',
                    'part_delta_eta', 'part_delta_phi', 'part_delta_R']:
        sorted_features[dtype]['j1'][feature] = []
        sorted_features[dtype]['j2'][feature] = []
        feature_array = [features[dtype]['j1'][feature],
                         features[dtype]['j2'][feature]]
        for i, array in enumerate(zip(*feature_array)):
            sorted_features[dtype]['j1'][feature].append(array[sort_index[i][0]])
            sorted_features[dtype]['j2'][feature].append(array[sort_index[i][1]])
        sorted_features[dtype]['j1'][feature] = sorted_features[dtype]['j1'][feature]
        sorted_features[dtype]['j2'][feature] = sorted_features[dtype]['j2'][feature]
    sorted_features[dtype] = ak.Array(sorted_features[dtype])
    # evaluate mjj
    j1_p4 = ak.zip({
    "pt"  : sorted_features[dtype]["j1"]["jet_pt"],
    "eta" : sorted_features[dtype]["j1"]["jet_eta"],
    "phi" : sorted_features[dtype]["j1"]["jet_phi"],
    "m"   : sorted_features[dtype]["j1"]["jet_m"]
    }, with_name="Momentum4D")
    j2_p4 = ak.zip({
        "pt"  : sorted_features[dtype]["j2"]["jet_pt"],
        "eta" : sorted_features[dtype]["j2"]["jet_eta"],
        "phi" : sorted_features[dtype]["j2"]["jet_phi"],
        "m"   : sorted_features[dtype]["j2"]["jet_m"]
    }, with_name="Momentum4D")
    jj_p4 = j1_p4.add(j2_p4)
    mjj = np.array(jj_p4.m)
    sorted_features[dtype]['jj'] = ak.Array({'mjj': mjj})

### Save features

Inclusive Region (without preselection)

In [ ]:
save_path = {
    "signal"     : os.path.join(save_dir, output_format, "col_based", f"point_cloud_W_qq_500_100_official.{output_format}"),
    "background" : os.path.join(save_dir, output_format, "col_based", f"point_cloud_QCD_qq.{output_format}")
}
for dtype in ['signal', 'background']:
    dirname = os.path.dirname(save_path[dtype])
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    save_fn(sorted_features[dtype], save_path[dtype])

Signal Region ( $3.3 < m_{jj} < 3.7$ GeV)

In [ ]:
save_path = {
    "signal"     : os.path.join(save_dir, output_format, "col_based", f"SR_point_cloud_W_qq_500_100_official.{output_format}"),
    "background" : os.path.join(save_dir, output_format, "col_based", f"SR_point_cloud_QCD_qq.{output_format}")
}
SR_sorted_features = {}
for dtype in ['signal', 'background']:
    mjj = sorted_features[dtype]['jj']['mjj']
    mask = np.where((mjj > 3.3) & (mjj < 3.7))
    SR_sorted_features[dtype] = sorted_features[dtype][mask]
    SR_sorted_features[dtype] = ak.Array(SR_sorted_features[dtype])
    save_fn(SR_sorted_features[dtype], save_path[dtype])

## Visualization

In [ ]:
# load back saved features
save_dir = "/pscratch/sd/c/chlcheng/dataset/anomaly_detection/LHC_Olympics_2020/LHCO_qq"
save_path = {
    "signal"     : os.path.join(save_dir, output_format, "col_based", f"SR_point_cloud_W_qq_500_100_official.{output_format}"),
    "background" : os.path.join(save_dir, output_format, "col_based", f"SR_point_cloud_QCD_qq.{output_format}")
}
features = {}
for dtype in save_path:
    features[dtype] = load_fn(save_path[dtype])

In [ ]:
dfs = {}
for dtype in ["signal", "background"]:
    data = {
        "lead_jet_pt"    : np.array(features[dtype]["j1"]["jet_pt"]),
        "lead_jet_eta"   : np.array(features[dtype]["j1"]["jet_eta"]),
        "lead_jet_phi"   : np.array(features[dtype]["j1"]["jet_phi"]),
        "lead_jet_m"     : np.array(features[dtype]["j1"]["jet_m"]),
        "lead_jet_N"     : np.array(features[dtype]["j1"]["N"]),
        "lead_jet_tau1"  : np.array(features[dtype]["j1"]["tau1"]),
        "lead_jet_tau2"  : np.array(features[dtype]["j1"]["tau2"]),
        "lead_jet_tau3"  : np.array(features[dtype]["j1"]["tau3"]),
        "lead_jet_tau12" : np.array(features[dtype]["j1"]["tau12"]),
        "lead_jet_tau23" : np.array(features[dtype]["j1"]["tau23"]),
        "sub_jet_pt"     : np.array(features[dtype]["j2"]["jet_pt"]),
        "sub_jet_eta"    : np.array(features[dtype]["j2"]["jet_eta"]),
        "sub_jet_phi"    : np.array(features[dtype]["j2"]["jet_phi"]),
        "sub_jet_m"      : np.array(features[dtype]["j2"]["jet_m"]),
        "sub_jet_N"      : np.array(features[dtype]["j2"]["N"]),
        "sub_jet_tau1"   : np.array(features[dtype]["j2"]["tau1"]),
        "sub_jet_tau2"   : np.array(features[dtype]["j2"]["tau2"]),
        "sub_jet_tau3"   : np.array(features[dtype]["j2"]["tau3"]),
        "sub_jet_tau12"  : np.array(features[dtype]["j2"]["tau12"]),
        "sub_jet_tau23"  : np.array(features[dtype]["j2"]["tau23"])
    }
    data['mjj'] =np.array(features[dtype]["jj"]["mjj"])
    dfs[dtype] = pd.DataFrame(data)

In [ ]:
from quickstats.plots import VariableDistributionPlot
import matplotlib.pyplot as plt
options = {
    'mjj': {
        'xlabel': '$m_{jj} [GeV]$',
        'bin_range': (3200, 3800),
        'variable_scale': 1000
    },
    'lead_jet_pt': {
        'xlabel': '$p_T^{j_1} [GeV]$',
        'bin_range': (1200, 2600),
        'variable_scale': 1000
    },
    'sub_jet_pt': {
        'xlabel': '$p_T^{j_2} [GeV]$',
        'bin_range': (1200, 2600),
        'variable_scale': 1000
    },
    'lead_jet_m': {
        'xlabel': '$m^{j_1} [GeV]$',
        'bin_range': (0, 800),
        'variable_scale': 1000
    },
    'sub_jet_m': {
        'xlabel': '$m^{j_2} [GeV]$',
        'bin_range': (0, 800),
        'variable_scale': 1000
    },
    'lead_jet_eta': {
        'xlabel': r'$\eta_{j_1}$'
    },
    'sub_jet_eta': {
        'xlabel': r'$\eta_{j_2}$'
    },
    'lead_jet_phi': {
        'xlabel': r'$\phi_{j_1}$'
    },
    'sub_jet_phi': {
        'xlabel': r'$\phi_{j_2}$'
    },
    'lead_jet_N': {
        'xlabel': '$N_{particle}^{j_1}$'
    },
    'sub_jet_N': {
        'xlabel': '$N_{particle}^{j_2}$'
    },
    'lead_jet_tau1': {
        'xlabel': r'$\tau_1^{j_1}$'
    },
    'sub_jet_tau1': {
        'xlabel': r'$\tau_1^{j_2}$'
    },
    'lead_jet_tau2': {
        'xlabel': r'$\tau_2^{j_1}$'
    },
    'sub_jet_tau2': {
        'xlabel': r'$\tau_2^{j_2}$'
    },
    'lead_jet_tau3': {
        'xlabel': r'$\tau_3^{j_1}$'
    },
    'sub_jet_tau3': {
        'xlabel': r'$\tau_3^{j_2}$'
    },
    'lead_jet_tau12': {
        'xlabel': r'$\tau_2^{j_1}/\tau_1^{j_1}$'
    },
    'sub_jet_tau12': {
        'xlabel': r'$\tau_2^{j_2}/\tau_1^{j_2}$'
    },
    'lead_jet_tau23': {
        'xlabel': r'$\tau_3^{j_1}/\tau_2^{j_1}$'
    },
    'sub_jet_tau23': {
        'xlabel': r'$\tau_3^{j_2}/\tau_2^{j_2}$'
    }
}
label_map = {
    'signal'    : 'Singal',
    'background': 'Background'
}
plotter = VariableDistributionPlot(dfs, label_map=label_map)
for column in options:
    plotter.draw(column,
                 **options[column],
                 bins=50,
                 normalize=True,
                 ypad=0.1)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from quickstats.plots import get_cmap
cmap_jet = get_cmap('jet')
cmap_jet.set_under('white')

def make_jet_image(features, title:str="jet", dimension:int=64,
                   xlabel:str="$\Delta \eta$", ylabel:str="$\Delta \phi$",
                   range=((-0.5, 0.5), (-0.5, 0.5)),
                   lognorm_range=(1e-4, 1),
                   cmap=cmap_jet):
    delta_eta = np.array(ak.flatten(features['part_delta_eta'], axis=None))
    delta_phi = np.array(ak.flatten(features['part_delta_phi'], axis=None))
    rel_pt = np.array(ak.flatten(features['part_rel_pt'], axis=None))
    hist2d, x_, y_ = np.histogram2d(delta_eta, 
                                    delta_phi,
                                    bins = (dimension, dimension),
                                    range=range,
                                    weights=rel_pt)
    # normalize the distribution
    hist2d_norm = hist2d / np.max(hist2d)
    vmin, vmax = lognorm_range
    fig, ax = plt.subplots(nrows=1, ncols=1)
    extend = np.array(range).flatten()
    handle = ax.imshow(hist2d_norm, cmap=cmap,
                       norm=LogNorm(vmin=vmin, vmax=vmax), extent=extend)
    fig.colorbar(handle, ax=ax)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)

In [ ]:
for dtype in ["signal", "background"]:
    for jtype, label in [("j1", "Leading Jet"), ("j2", "Subleading Jet")]:
        make_jet_image(sorted_features[dtype][jtype], title = f"{dtype.title()}: {label}")
        plt.show()

## Additional Background Sample

In [ ]:
input_path = os.path.join(proj_dir, "events_anomalydetection_qcd_extra_inneronly_4vecs_and_features.h5")
df = pd.read_hdf(input_path)

In [ ]:
print(f'Number of events: {df.shape[0]}')
df.head()

### Jet Clustering

In [ ]:
from aliad.interface.fastjet import JetClusteringTool
tool = JetClusteringTool(R=1.0, ptmin=20.0)

In [ ]:
jet_constituents = {
    "extra_background": tool.get_constituent_arrays_from_padded_data(df[df.columns[:2100]].values)
}

In [ ]:
# need to process in batches due to high memory demand

save_dir = "/pscratch/sd/c/chlcheng/dataset/anomaly_detection/LHC_Olympics_2020/LHCO_qq"

save_path = {
    "extra_background" : os.path.join(save_dir, output_format, "row_based", "batches", "point_cloud_extra_bkg_{obj}_batch_{i}." + output_format)
}
if not os.path.exists(os.path.dirname(save_path['extra_background'])):
    os.makedirs(os.path.dirname(save_path['extra_background']))

batchsize = 100000
# maximum number of jets to save the constituent information
max_jet = 4
cache = True
save = True
all_jets = {}
all_constituents = {}
for dtype in ["extra_background"]:
    all_jets[dtype] = []
    all_constituents[dtype] = {}
    for j in range(max_jet):
        all_constituents[dtype][f'j{j+1}'] = []
    nbatch = len(jet_constituents[dtype]) // batchsize
    if len(jet_constituents[dtype]) % batchsize != 0:
        nbatch += 1
    for i in range(nbatch):
        i_start = i * batchsize
        i_end  = max((i+1)*batchsize, nbatch)
        path_jets = save_path[dtype].format(i=i+1, obj="jets")
        paths_constituents = []
        for j in range(max_jet):
            path_constituents = save_path[dtype].format(i=i+1, obj=f"constituents_jet_{j+1}")
            paths_constituents.append(path_constituents)
        # only do jet clustering if not cached
        if os.path.exists(path_jets) and all([os.path.exists(path) for path in paths_constituents]) and cache:
            jets_batch = None
        else:
            print(f'Creating clustered jet data for "{dtype}" (batch {i+1} / {nbatch})')
            jets_batch = tool.get_inclusive_jets_array(jet_constituents[dtype][i_start: i_end])
        # convert to awkward array
        if os.path.exists(path_jets) and cache:
            print(f'Cached existing clustered jet data from "{path_jets}"')
            jet_array = load_fn(path_jets)
        else:
            print(f'Converting clustered jet data to awkward arrays')
            jet_array = tool.to_awkward_jets_array(jets_batch)
            if save:
                print(f'Saving clustered jet data to "{path_jets}"')
                save_fn(jet_array, path_jets)
        all_jets[dtype].append(jet_array)
        for j in range(max_jet):
            path_constituents = paths_constituents[j]
            if os.path.exists(path_constituents) and cache:
                print(f'Cached existing clustered jet constituents data from "{path_constituents}"')
                constituent_array = load_fn(path_constituents)
            else:
                print(f'Creating clustered jet constituents data for "{dtype}" (jet {j + 1})')
                constituent_array = tool.to_awkward_constituents_array(jets_batch, jet_index=j)
                if save:
                    print(f'Saving clustered jet constituents data to "{path_constituents}"')
                    save_fn(constituent_array, path_constituents)
            all_constituents[dtype][f'j{j+1}'].append(constituent_array)
    all_jets[dtype] = ak.concatenate(all_jets[dtype])
    for j in range(max_jet):
        all_constituents[dtype][f'j{j+1}'] = ak.concatenate(all_constituents[dtype][f'j{j+1}'])

In [ ]:
# save the combined data
save_path = {
    "extra_background" : os.path.join(save_dir, output_format, "row_based", "batches", "point_cloud_extra_bkg_{obj}." + output_format)
}
for dtype in ['extra_background']:
    save_fn(all_jets[dtype], save_path[dtype].format(obj='jets'))
    for j in range(max_jet):
        save_fn(all_constituents[dtype][f'j{j+1}'], save_path[dtype].format(obj=f'constituents_jet_{j+1}'))

In [ ]:
features = {}
for dtype in ["extra_background"]:
    features[dtype] = {}
    for key in all_constituents[dtype]:
        features[dtype][key] = {}
        # jet features
        jet_p4 = ak.sum(all_constituents[dtype][key], axis=-1)
        # convert to TeV
        features[dtype][key]["jet_pt"]   = jet_p4.pt / 1000.
        features[dtype][key]["jet_eta"]  = jet_p4.eta
        features[dtype][key]["jet_phi"]  = jet_p4.phi
        features[dtype][key]["jet_m"]    = jet_p4.m / 1000.
        features[dtype][key]["N"]        = ak.count(all_constituents[dtype][key], axis=-1)
        # particle features
        features[dtype][key]['part_pt']        = all_constituents[dtype][key].pt / 1000.
        features[dtype][key]['part_eta']       = all_constituents[dtype][key].eta
        features[dtype][key]['part_phi']       = all_constituents[dtype][key].phi
        features[dtype][key]['part_e']         = all_constituents[dtype][key].e / 1000.
        features[dtype][key]['part_rel_pt']    = all_constituents[dtype][key].pt / jet_p4.pt
        features[dtype][key]['part_delta_eta'] = all_constituents[dtype][key].deltaeta(jet_p4)
        features[dtype][key]['part_delta_phi'] = all_constituents[dtype][key].deltaphi(jet_p4)
        features[dtype][key]['part_delta_R']   = all_constituents[dtype][key].deltaR(jet_p4)

In [ ]:
for dtype in ["extra_background"]:
    for jtype in ['j1', 'j2']:
        for tau in ['tau1', 'tau2', 'tau3']:
            features[dtype][jtype][f'{tau}'] = df[f'{tau}{jtype}'].values
        ## can use any of tau1, tau2 or tau3 to check for zeros
        mask = ((features[dtype][jtype][f'tau1'] > 0) & (features[dtype][jtype][f'tau2'] > 0))
        features[dtype][jtype][f'tau12'] = np.where(mask, np.divide(features[dtype][jtype][f'tau2'],
                                                                    features[dtype][jtype][f'tau1'], where=mask), 0)
        features[dtype][jtype][f'tau23'] = np.where(mask, np.divide(features[dtype][jtype][f'tau3'],
                                                                    features[dtype][jtype][f'tau2'], where=mask), 0)
        ## remove events with undefined nsubjetiness ratio
        #for feature in features[dtype][jtype]:
        #    features[dtype][jtype][feature] = features[dtype][jtype][feature][mask]
        #features[dtype][jtype][f'tau12'] = features[dtype][jtype][f'tau2'] / features[dtype][jtype][f'tau1']
        #features[dtype][jtype][f'tau23'] = features[dtype][jtype][f'tau3'] / features[dtype][jtype][f'tau2']

In [ ]:
sorted_features = {}
for dtype in ['extra_background']:
    sorted_features[dtype] = {'j1':{}, 'j2':{}}
    mass_array = [features[dtype]['j1']['jet_m'],
                  features[dtype]['j2']['jet_m']]
    mass_array = ak.to_numpy(ak.Array(mass_array)).transpose()
    sort_index = np.argsort(-mass_array, axis=1)
    for feature in ['jet_pt', 'jet_eta', 'jet_phi', 'jet_m', 'N', 'tau1', 'tau2', 'tau3', 'tau12', 'tau23']:
        feature_array = [features[dtype]['j1'][feature],
                         features[dtype]['j2'][feature]]
        feature_array = ak.to_numpy(ak.Array(feature_array)).transpose()
        feature_array = np.take_along_axis(feature_array, sort_index, axis=1)
        sorted_features[dtype]['j1'][feature] = np.ascontiguousarray(feature_array[:, 0])
        sorted_features[dtype]['j2'][feature] = np.ascontiguousarray(feature_array[:, 1])
    for feature in ['part_pt', 'part_eta', 'part_phi', 'part_e', 'part_rel_pt',
                    'part_delta_eta', 'part_delta_phi', 'part_delta_R']:
        sorted_features[dtype]['j1'][feature] = []
        sorted_features[dtype]['j2'][feature] = []
        feature_array = [features[dtype]['j1'][feature],
                         features[dtype]['j2'][feature]]
        for i, array in enumerate(zip(*feature_array)):
            sorted_features[dtype]['j1'][feature].append(array[sort_index[i][0]])
            sorted_features[dtype]['j2'][feature].append(array[sort_index[i][1]])
        sorted_features[dtype]['j1'][feature] = sorted_features[dtype]['j1'][feature]
        sorted_features[dtype]['j2'][feature] = sorted_features[dtype]['j2'][feature]
    sorted_features[dtype] = ak.Array(sorted_features[dtype])
    # evaluate mjj
    j1_p4 = ak.zip({
    "pt"  : sorted_features[dtype]["j1"]["jet_pt"],
    "eta" : sorted_features[dtype]["j1"]["jet_eta"],
    "phi" : sorted_features[dtype]["j1"]["jet_phi"],
    "m"   : sorted_features[dtype]["j1"]["jet_m"]
    }, with_name="Momentum4D")
    j2_p4 = ak.zip({
        "pt"  : sorted_features[dtype]["j2"]["jet_pt"],
        "eta" : sorted_features[dtype]["j2"]["jet_eta"],
        "phi" : sorted_features[dtype]["j2"]["jet_phi"],
        "m"   : sorted_features[dtype]["j2"]["jet_m"]
    }, with_name="Momentum4D")
    jj_p4 = j1_p4.add(j2_p4)
    mjj = np.array(jj_p4.m)
    sorted_features[dtype]['jj'] = ak.Array({"mjj": mjj})

In [ ]:
save_path = {
    "extra_background" : os.path.join(save_dir, output_format, "col_based", f"point_cloud_extra_QCD_qq.{output_format}")
}
for dtype in ['extra_background']:
    save_fn(sorted_features[dtype], save_path[dtype])

In [ ]:
save_path = {
    "extra_background" : os.path.join(save_dir, output_format, "col_based", f"SR_point_cloud_extra_QCD_qq.{output_format}")
}
SR_sorted_features = {}
for dtype in ['extra_background']:
    mjj = sorted_features[dtype]['jj']['mjj']
    mask = np.where((mjj > 3.3) & (mjj < 3.7))
    SR_sorted_features[dtype] = sorted_features[dtype][mask]
    SR_sorted_features[dtype] = ak.Array(SR_sorted_features[dtype])
    save_fn(SR_sorted_features[dtype], save_path[dtype])

## Visualization

In [ ]:
# load back saved features
save_dir = "/pscratch/sd/c/chlcheng/dataset/anomaly_detection/LHC_Olympics_2020/LHCO_qq"
save_path = {
    "signal"     : os.path.join(save_dir, output_format, "col_based", f"SR_point_cloud_W_qq_500_100_official.{output_format}"),
    "background" : os.path.join(save_dir, output_format, "col_based", f"SR_point_cloud_QCD_qq.{output_format}"),
    "extra_background" : os.path.join(save_dir, output_format, "col_based", f"SR_point_cloud_extra_QCD_qq.{output_format}")
}
features = {}
for dtype in save_path:
    features[dtype] = load_fn(save_path[dtype])

In [ ]:
dfs = {}
for dtype in features:
    data = {
        "lead_jet_pt"    : np.array(features[dtype]["j1"]["jet_pt"]),
        "lead_jet_eta"   : np.array(features[dtype]["j1"]["jet_eta"]),
        "lead_jet_phi"   : np.array(features[dtype]["j1"]["jet_phi"]),
        "lead_jet_m"     : np.array(features[dtype]["j1"]["jet_m"]),
        "lead_jet_N"     : np.array(features[dtype]["j1"]["N"]),
        "lead_jet_tau1"  : np.array(features[dtype]["j1"]["tau1"]),
        "lead_jet_tau2"  : np.array(features[dtype]["j1"]["tau2"]),
        "lead_jet_tau3"  : np.array(features[dtype]["j1"]["tau3"]),
        "lead_jet_tau12" : np.array(features[dtype]["j1"]["tau12"]),
        "lead_jet_tau23" : np.array(features[dtype]["j1"]["tau23"]),
        "sub_jet_pt"     : np.array(features[dtype]["j2"]["jet_pt"]),
        "sub_jet_eta"    : np.array(features[dtype]["j2"]["jet_eta"]),
        "sub_jet_phi"    : np.array(features[dtype]["j2"]["jet_phi"]),
        "sub_jet_m"      : np.array(features[dtype]["j2"]["jet_m"]),
        "sub_jet_N"      : np.array(features[dtype]["j2"]["N"]),
        "sub_jet_tau1"   : np.array(features[dtype]["j2"]["tau1"]),
        "sub_jet_tau2"   : np.array(features[dtype]["j2"]["tau2"]),
        "sub_jet_tau3"   : np.array(features[dtype]["j2"]["tau3"]),
        "sub_jet_tau12"  : np.array(features[dtype]["j2"]["tau12"]),
        "sub_jet_tau23"  : np.array(features[dtype]["j2"]["tau23"])
    }
    data['mjj'] =np.array(features[dtype]["jj"]["mjj"])
    dfs[dtype] = pd.DataFrame(data)

In [ ]:
from quickstats.plots import VariableDistributionPlot
import matplotlib.pyplot as plt
options = {
    'mjj': {
        'xlabel': '$m_{jj} [GeV]$',
        'bin_range': (3200, 3800),
        'variable_scale': 1000
    },
    'lead_jet_pt': {
        'xlabel': '$p_T^{j_1} [GeV]$',
        'bin_range': (1200, 2600),
        'variable_scale': 1000
    },
    'sub_jet_pt': {
        'xlabel': '$p_T^{j_2} [GeV]$',
        'bin_range': (1200, 2600),
        'variable_scale': 1000
    },
    'lead_jet_m': {
        'xlabel': '$m^{j_1} [GeV]$',
        'bin_range': (0, 800),
        'variable_scale': 1000
    },
    'sub_jet_m': {
        'xlabel': '$m^{j_2} [GeV]$',
        'bin_range': (0, 800),
        'variable_scale': 1000
    },
    'lead_jet_eta': {
        'xlabel': r'$\eta_{j_1}$'
    },
    'sub_jet_eta': {
        'xlabel': r'$\eta_{j_2}$'
    },
    'lead_jet_phi': {
        'xlabel': r'$\phi_{j_1}$'
    },
    'sub_jet_phi': {
        'xlabel': r'$\phi_{j_2}$'
    },
    'lead_jet_N': {
        'xlabel': '$N_{particle}^{j_1}$'
    },
    'sub_jet_N': {
        'xlabel': '$N_{particle}^{j_2}$'
    },
    'lead_jet_tau1': {
        'xlabel': r'$\tau_1^{j_1}$'
    },
    'sub_jet_tau1': {
        'xlabel': r'$\tau_1^{j_2}$'
    },
    'lead_jet_tau2': {
        'xlabel': r'$\tau_2^{j_1}$'
    },
    'sub_jet_tau2': {
        'xlabel': r'$\tau_2^{j_2}$'
    },
    'lead_jet_tau3': {
        'xlabel': r'$\tau_3^{j_1}$'
    },
    'sub_jet_tau3': {
        'xlabel': r'$\tau_3^{j_2}$'
    },
    'lead_jet_tau12': {
        'xlabel': r'$\tau_2^{j_1}/\tau_1^{j_1}$'
    },
    'sub_jet_tau12': {
        'xlabel': r'$\tau_2^{j_2}/\tau_1^{j_2}$'
    },
    'lead_jet_tau23': {
        'xlabel': r'$\tau_3^{j_1}/\tau_2^{j_1}$'
    },
    'sub_jet_tau23': {
        'xlabel': r'$\tau_3^{j_2}/\tau_2^{j_2}$'
    }
}
label_map = {
    'signal'    : 'Singal',
    'background': 'Background',
    'extra_background': 'Extra Background'
}
plotter = VariableDistributionPlot(dfs, label_map=label_map)
for column in options:
    plotter.draw(column,
                 **options[column],
                 bins=50,
                 normalize=True,
                 ypad=0.1)
    plt.show()